In [1]:
import requests 
from bs4 import BeautifulSoup

In [2]:
url = "https://www.climatecouncil.org.au/resources/australia-key-role-to-play-clean-energy-reshapes-indo-pacific-relations/"
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'}
response = requests.get(url, headers=headers)

In [3]:
soup=BeautifulSoup(response.content, "html.parser")

In [4]:
print(soup.prettify())

<!DOCTYPE html>
<!--[if lt IE 7]>      <html class="no-js lt-ie9 lt-ie8 lt-ie7" lang="en-GB"> <![endif]-->
<!--[if IE 7]>         <html class="no-js lt-ie9 lt-ie8" lang="en-GB"> <![endif]-->
<!--[if IE 8]>         <html class="no-js lt-ie9" lang="en-GB"> <![endif]-->
<!--[if gt IE 8]><!-->
<html class="no-js" lang="en-GB">
 <!--<![endif]-->
 <head>
  <meta charset="utf-8"/>
  <meta content="width=device-width, initial-scale=1.0, maximum-scale=1.0, user-scalable=0" name="viewport"/>
  <link href="http://gmpg.org/xfn/11" rel="profile"/>
  <link href="https://www.climatecouncil.org.au/xmlrpc.php" rel="pingback"/>
  <link href="https://www.climatecouncil.org.au/wp-content/themes/climate-council/assets/dist/img/favicon.png" rel="shortcut icon" type="image/png"/>
  <link href="//cdn.jsdelivr.net/jquery.slick/1.6.0/slick.css" rel="stylesheet" type="text/css">
   <meta content="index, follow, max-image-preview:large, max-snippet:-1, max-video-preview:-1" name="robots">
    <!-- Google Tag Mana

In [5]:
main_content = soup.find('article')
for element in main_content(['header', 'footer', 'aside', 'script']):
    element.decompose()

In [6]:
text_list = []
stop_text = "The time is right for Australia to accelerate clean energy and associated climate action and secure its economic and strategic advantage in the Indo-Pacific’s clean energy future."
for paragraph in main_content.find_all('p'):
    paragraph_text = paragraph.get_text(separator=' ', strip=False)

    if stop_text in paragraph_text:
        text_list.append(paragraph_text)
        break
    text_list.append(paragraph_text)

In [7]:
article_text = '\n'.join(text_list)

In [8]:
print(article_text)

With the incoming federal government declaring it will make Australia a “renewable superpower”, China dominating clean energy supply chains, and the Russian war in Ukraine disrupting the global energy market, now is a pivotal moment for Australia to shape the future of clean energy within the Indo-Pacific.
A  new report by the Perth USAsia Centre, in collaboration with the Climate Council , recommends five practical actions for Australia to secure its economic and strategic clean energy advantages in the Indo-Pacific :
Published ahead of a major international forum on energy supply chains – the  Sydney Energy Forum  (12-13 July) – the report,  Reenergising Indo-Pacific Relations: Australia’s Clean Energy Opportunity ,  explains that the Indo-Pacific sits at the heart of the global shift from fossil fuels to clean energy systems, the ramifications for Australia’s economic and strategic interests, and the leading role that our country could play.
Global warming is a significant security 

In [9]:
pip install nltk

Note: you may need to restart the kernel to use updated packages.


In [10]:
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer, WordNetLemmatizer

In [11]:
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package punkt to /Users/loogyee/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/loogyee/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /Users/loogyee/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /Users/loogyee/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [12]:
article_text = article_text.lower()

In [13]:
tokens = word_tokenize(article_text)

In [14]:
tokens = [re.sub(r'[^\w\s]', '', token) for token in tokens]

In [15]:
stop_words = set(stopwords.words('english'))
tokens = [word for word in tokens if word not in stop_words]

In [16]:
stemmer = PorterStemmer()
stemmed_tokens = [stemmer.stem(word) for word in tokens]

In [17]:
lemmatizer = WordNetLemmatizer()
lemmatized_tokens = [lemmatizer.lemmatize(word) for word in tokens]

In [18]:
print(lemmatized_tokens)

['incoming', 'federal', 'government', 'declaring', 'make', 'australia', '', 'renewable', 'superpower', '', '', 'china', 'dominating', 'clean', 'energy', 'supply', 'chain', '', 'russian', 'war', 'ukraine', 'disrupting', 'global', 'energy', 'market', '', 'pivotal', 'moment', 'australia', 'shape', 'future', 'clean', 'energy', 'within', 'indopacific', '', 'new', 'report', 'perth', 'usasia', 'centre', '', 'collaboration', 'climate', 'council', '', 'recommends', 'five', 'practical', 'action', 'australia', 'secure', 'economic', 'strategic', 'clean', 'energy', 'advantage', 'indopacific', '', 'published', 'ahead', 'major', 'international', 'forum', 'energy', 'supply', 'chain', '', 'sydney', 'energy', 'forum', '', '1213', 'july', '', '', 'report', '', 'reenergising', 'indopacific', 'relation', '', 'australia', '', 'clean', 'energy', 'opportunity', '', 'explains', 'indopacific', 'sits', 'heart', 'global', 'shift', 'fossil', 'fuel', 'clean', 'energy', 'system', '', 'ramification', 'australia', '',

In [19]:
pip install spacy

Note: you may need to restart the kernel to use updated packages.


In [20]:
pip install https://github.com/explosion/spacy-models/releases/download/en_core_web_sm-3.6.0/en_core_web_sm-3.6.0.tar.gz

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 13.2 MB/s eta 0:00:0000:0100:01
  Preparing metadata (setup.py) ... done
Note: you may need to restart the kernel to use updated packages.


In [21]:
import spacy
nlp = spacy.load("en_core_web_sm")

In [22]:
doc = nlp(article_text)
unique_entities = set()

for ent in doc.ents:
    unique_entities.add((ent.text, ent.label_))
for entity, label in unique_entities:
    print(f"{entity} - {label}")

dr wesley - PERSON
the russian war - EVENT
china - GPE
australian - NORP
the perth usasia centre - ORG
five - CARDINAL
australia - GPE
12-13 july - DATE
james bowen - PERSON
japan - GPE
korea - GPE
ukraine - GPE


In [23]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [24]:
tfidf_vectorizer = TfidfVectorizer()
X = tfidf_vectorizer.fit_transform([' '.join(lemmatized_tokens)])

In [25]:
print(X.shape)

(1, 172)


In [26]:
from spacy import displacy

In [27]:
nlp = spacy.load("en_core_web_sm")

In [29]:
doc = nlp(article_text)

In [30]:
displacy.render(doc, style="dep", jupyter=True)

In [32]:
important_deps = ["nsubj", "dobj", "ROOT", "xcomp", "ccomp", "conj", "amod"]
for token in doc:
    if token.dep_ in important_deps:
        print(f"{token.text} --> {token.dep_} --> {token.head.text}")

incoming --> amod --> government
federal --> amod --> government
government --> nsubj --> declaring
it --> nsubj --> make
make --> ccomp --> declaring
australia --> nsubj --> superpower
renewable --> amod --> superpower
superpower --> ccomp --> make
china --> nsubj --> is
clean --> amod --> chains
chains --> dobj --> dominating
russian --> amod --> war
war --> conj --> china
global --> amod --> market
market --> dobj --> disrupting
is --> ROOT --> is
pivotal --> amod --> moment
australia --> nsubj --> shape
future --> dobj --> shape
clean --> amod --> energy
new --> amod --> report
report --> nsubj --> recommends
recommends --> ccomp --> explains
practical --> amod --> actions
actions --> dobj --> recommends
australia --> nsubj --> secure
economic --> amod --> clean
strategic --> conj --> economic
clean --> amod --> advantages
advantages --> dobj --> secure
major --> amod --> forum
international --> amod --> forum
forum --> dobj --> published
relations --> dobj --> reenergising
clean -

In [33]:
sentence_spans = list(doc.sents)
displacy.render(sentence_spans, style="ent", jupyter=True)

/Users/loogyee/anaconda3/lib/python3.11/site-packages/spacy/displacy/__init__.py:213: UserWarning: [W006] No entities to visualize found in Doc object. If this is surprising to you, make sure the Doc was processed using a model that supports named entity recognition, and check the `doc.ents` property manually if necessary.
  warnings.warn(Warnings.W006)
